<h1>Table of Contents<span class="tocSkip"></span></h1>
<div class="toc"><ul class="toc-item"></ul></div>

In [0]:
!mkdir datawav 
!mkdir dataref

In [4]:
import tensorflow as tf
import numpy as np
import scipy as scp
from scipy import signal, linalg
# Solve Toeplitz Wiener
from scipy.linalg import solve_toeplitz
from numpy import asarray, array, ravel, repeat, prod, mean, where, ones
#!pip install scikits.audiolab
#!pip install --upgrade-strategy=only-if-needed git+https://github.com/Uiuran/BregmanToolkit
#!pip install scikit-image
#import scikits.audiolab as audio
import matplotlib.pyplot as plt
from tensorflow.python.client import timeline
correlate = signal.correlate
import os
import librosa


# Tensorboard display
from IPython.display import clear_output, Image, display, HTML

In [2]:
def freeze_session(session, keep_var_names=None, output_names=None, clear_devices=True):
    """
    Freezes the state of a session into a pruned computation graph.

    Creates a new computation graph where variable nodes are replaced by
    constants taking their current value in the session. The new graph will be
    pruned so subgraphs that are not necessary to compute the requested
    outputs are removed.    
    
    Parameters:
    
    - session: The TensorFlow session to be frozen.
    
    Default Keyword Parameters
    
    - keep_var_names: A list of variable names that should not be frozen. Defaults None.     
    - output_names: Names of the relevant graph outputs/operation/tensor to be written. Defaults None.
    - clear_devices: Remove the device directives from the graph for better portability. Defaults True.
    
    return The frozen graph definition.
    """    
    graph = session.graph
    with graph.as_default():
        freeze_var_names = list(set(v.op.name for v in tf.global_variables()).difference(keep_var_names or []))
        output_names = output_names or []
        output_names += [v.op.name for v in tf.global_variables()]
        # Graph -> GraphDef ProtoBuf
        input_graph_def = graph.as_graph_def()
        if clear_devices:
            for node in input_graph_def.node:
                node.device = ""
        frozen_graph = convert_variables_to_constants(session, input_graph_def,
                                                      output_names, freeze_var_names)
        return frozen_graph

def wiener(im, mysize=None, noise=None):
    """
    Perform a Wiener filter on an N-dimensional array.
    Apply a Wiener filter to the N-dimensional array `im`.
    Parameters
    ----------
    im : ndarray
        An N-dimensional array.
    mysize : int or array_like, optional
        A scalar or an N-length list giving the size of the Wiener filter
        window in each dimension.  Elements of mysize should be odd.
        If mysize is a scalar, then this scalar is used as the size
        in each dimension.
    noise : float, optional
        The noise-power to use. If None, then noise is estimated as the
        average of the local variance of the input.
    Returns
    -------
    out : ndarray
        Wiener filtered result with the same shape as `im`.
    """      
    
      
    
    im = asarray(im)
    if mysize is None:
        mysize = [3] * im.ndim
    mysize = asarray(mysize)
    if mysize.shape == ():
        mysize = repeat(mysize.item(), im.ndim)

    # Estimate the local mean
    lMean = correlate(im, ones(mysize), 'same') / prod(mysize, axis=0)

    # Estimate the local variance
    lVar = (correlate(im ** 2, ones(mysize), 'same') /
            prod(mysize, axis=0) - lMean ** 2+1e-8)

    # Estimate the noise power if needed.
    if noise is None:
        noise = mean(ravel(lVar), axis=0)

    res = (im - lMean)
    res *= (1 - noise / lVar)
    res += lMean
    out = where(lVar < noise, lMean, res)    
    subtract = im-out
    return out,subtract

In [2]:
# funções auxiliares
def strip_consts(graph_def, max_const_size=32):
    """Strip large constant values from graph_def."""
    strip_def = tf.GraphDef()
    for n0 in graph_def.node:
        n = strip_def.node.add() 
        n.MergeFrom(n0)
        if n.op == 'Const':
            tensor = n.attr['value'].tensor
            size = len(tensor.tensor_content)
            if size > max_const_size:
                tensor.tensor_content = tf.compat.as_bytes("<stripped %d bytes>"%size)
    return strip_def
  
def rename_nodes(graph_def, rename_func):
    res_def = tf.GraphDef()
    for n0 in graph_def.node:
        n = res_def.node.add() 
        n.MergeFrom(n0)
        n.name = rename_func(n.name)
        for i, s in enumerate(n.input):
            n.input[i] = rename_func(s) if s[0]!='^' else '^'+rename_func(s[1:])
    return res_def
  
# Função que usa HTML e javascript para exibir tensorboar no notebook e web
def show_graph(graph_def, max_const_size=32):
    """Visualize TensorFlow graph."""
    if hasattr(graph_def, 'as_graph_def'):
        graph_def = graph_def.as_graph_def()
    strip_def = strip_consts(graph_def, max_const_size=max_const_size)
    code = """
        <script>
          function load() {{
            document.getElementById("{id}").pbtxt = {data};
          }}
        </script>
        <link rel="import" href="https://tensorboard.appspot.com/tf-graph-basic.build.html" onload=load()>
        <div style="height:600px">
          <tf-graph-basic id="{id}"></tf-graph-basic>
        </div>
    """.format(data=repr(str(strip_def)), id='graph'+str(np.random.rand()))
  
    iframe = """
        <iframe seamless style="width:800px;height:620px;border:0" srcdoc="{}"></iframe>
    """.format(code.replace('"', '&quot;'))
    display(HTML(iframe))

In [7]:
#files = [f for f in os.listdir('/content/datawav')]
refs = [f for f in os.listdir('/content/dataref')]
data,fs,enc = audio.wavread('/content/dataref/'+refs[0])
l = np.size(data)

OSError: [Errno 2] No such file or directory: '/content/dataref'

In [0]:
tensor_data = np.ndarray(shape=(1,l,1))
tensor_ref = np.ndarray(shape=(5,l,1))

for r in range(len(refs)):  
  
  data1,_,_ = audio.wavread('/content/dataref/'+refs[r])
  tensor_ref[r,:,0] = data1.copy()
  
for f in range(len(files[1:2])):    
  data,_,_ = audio.wavread('/content/datawav/'+files[f])
  tensor_data[f,:,0] = data.copy()

print(files[1:2])

['0EAFC8DC-9EC3-42CE-A969-CE4880A0123E.wav']


In [9]:

# Comp. Graph 
graph = tf.Graph()
with graph.as_default():
  signal_in = tf.placeholder(tf.float32,(None,16000,1), name='signal_in')  
  filter = tf.get_variable('filter', shape=[8000,1,1],initializer=tf.random_normal_initializer(), dtype=tf.float32)
  w = tf.get_variable('w', shape=[5,16000],initializer=tf.random_normal_initializer(), dtype=tf.float32)
  signal_ref = tf.placeholder(tf.float32,(None,16000,1), name='signal_ref')
  # 1D Convolve  which internally uses 2D reshaped https://www.tensorflow.org/api_docs/python/tf/nn/conv1d
  signal_out = tf.nn.conv1d(signal_in,filter,1,'SAME', name='signal_out')  
  loss = tf.reduce_sum(tf.math.squared_difference(signal_out,tf.tensordot(w,signal_ref,[[0,1],[0,1]], name='ref_contraction'), name = 'squared'), name = 'loss')
  minimize_op = tf.train.AdamOptimizer(learning_rate=0.005).minimize(-loss)
  # Print operations for the graph built

  for op in graph.get_operations():
    print(op.name)
  graph_def = graph.as_graph_def()
  show_graph(graph_def)


signal_in
filter/Initializer/random_normal/shape
filter/Initializer/random_normal/mean
filter/Initializer/random_normal/stddev
filter/Initializer/random_normal/RandomStandardNormal
filter/Initializer/random_normal/mul
filter/Initializer/random_normal
filter
filter/Assign
filter/read
w/Initializer/random_normal/shape
w/Initializer/random_normal/mean
w/Initializer/random_normal/stddev
w/Initializer/random_normal/RandomStandardNormal
w/Initializer/random_normal/mul
w/Initializer/random_normal
w
w/Assign
w/read
signal_ref
signal_out/ExpandDims/dim
signal_out/ExpandDims
signal_out/ExpandDims_1/dim
signal_out/ExpandDims_1
signal_out
signal_out/Squeeze
ref_contraction/transpose/perm
ref_contraction/transpose
ref_contraction/Reshape/shape
ref_contraction/Reshape
ref_contraction/axes
ref_contraction/free
ref_contraction/Shape
ref_contraction/GatherV2/axis
ref_contraction/GatherV2
ref_contraction/GatherV2_1/axis
ref_contraction/GatherV2_1
ref_contraction/Const
ref_contraction/Prod
ref_contractio

In [11]:
# Create session

with graph.as_default():
  session = tf.Session()

  feed_dict = {
      signal_in: tensor_data,
      signal_ref:tensor_ref   
  }
  options = tf.RunOptions(trace_level=tf.RunOptions.FULL_TRACE)
  run_metadata = tf.RunMetadata()
  session.run(tf.global_variables_initializer())
  lossval = np.zeros(140)
  # Perform  gradient descent steps    
  for step in range(3):
    
    loss_value = session.run(loss, feed_dict)    
    lossval[step] = loss_value

    if step % 1 == 0:
      print("Step:", step, " Loss:", loss_value)
      if step % 5 == 0 and step != 0:
        loss_diff = np.diff(lossval[np.nonzero(lossval)])
        print("Mean Loss Growth ",np.mean(loss_diff) )
    
      
    session.run(minimize_op,
                feed_dict = feed_dict,
                options=options,
                run_metadata=run_metadata)    
    
    
    # Profiling
    #fetched_timeline = timeline.Timeline(run_metadata.step_stats)
    #chrome_trace = fetched_timeline.generate_chrome_trace_format()
    #with open('timeline_0_0_step_%d.json' % step, 'w') as f:
      #f.write(chrome_trace)

# Plot

  signal_out_value = session.run(graph.get_tensor_by_name('signal_out/Squeeze:0'), feed_dict)  
  filter_value = filter.eval(session=session)
  print('Wiener filter ')
  plt.figure()
  plt.plot(filter_value[:,0,0])    
  print('output_filter_SGD')
  plt.figure()
  plt.plot(signal_out_value[0,:,0])  
  print('input_signal')  
  plt.figure()
  plt.plot(tensor_data[0,:,0])

  audio.wavwrite( signal_out_value[0,:,0]/(np.std(signal_out_value[0,:,0],ddof=1.0)+1e-10),'/content/max_0_12072019.wav', fs=fs, enc=enc)
  
  frozen_graph = freeze_session(session, output_names= None)
  tf.train.write_graph(frozen_graph, "model", "./wiener.pb", as_text=False)   

NameError: name 'tensor_data' is not defined

In [0]:
#audio.wavwrite(signal_out_value[0,:,0],'/content/noiseout8000.wav', fs=fs, enc=enc)
audio.wavwrite( signal_out_value[0,:,0]-mu ,'/content/maximin.wav', fs=fs, enc=enc)
print(files[1:2])
'''
  filter_value = filter.eval(session=session)
  print('Wiener filter ')
  plt.figure()
  plt.plot(filter_value[0,:,0])    
  print('output_filter_SGD')
  plt.figure()
  plt.plot(signal_out_value[0,:,0])  
  print('input_signal')  
  plt.figure()
  plt.plot(tensor_data[0,:,0])
'''

['E3774329-5939-4585-BA17-88CA08330C5E.wav']


"\n  filter_value = filter.eval(session=session)\n  print('Wiener filter ')\n  plt.figure()\n  plt.plot(filter_value[0,:,0])    \n  print('output_filter_SGD')\n  plt.figure()\n  plt.plot(signal_out_value[0,:,0])  \n  print('input_signal')  \n  plt.figure()\n  plt.plot(tensor_data[0,:,0])\n"

In [0]:
#audio.wavwrite( signal_out_value[0,:,0]/(np.std(signal_out_value[0,:,0],ddof=1.0)+1e-10),'/content/max_0_12072019.wav', fs=fs, enc=enc)
audio.wavwrite( 10.0*tensor_data[0,:,0],'/content/0EAFC8DC-9EC3-42CE-A969-CE4880A0123E.wav', fs=fs, enc=enc)

In [0]:
measure,fs,enc = audio.wavread('/content/birds.wav')
airnoise,fs1,enc = audio.wavread('/content/ID 1.477704.wav')

In [0]:
def downsampling(data,num):
  '''
   Downsampling por num(int) feito como se fosse no C
  '''
  d = np.copy(data)
  j=0
  for i in range(len(d)):

    if i%int(num) == 0:
      j +=1
    else:
      d[i] = 0.0
  datadown = np.zeros(j)
  j=0
  for i in range(len(d)):
  
    if d[i] != 0.0: 
      #print(d[i])
      datadown[j] = d[i]
      j += 1  

  return datadown

def amostragem(data, fs, plot = False):
  '''
   Amostragem de downsampling de uma serie temporal segundo Nyquist-Shannon. Alem disso o numero maximo
   de frequencias com o spectro calculado por ciclo deve ser igual ao numero de pontos N/2 (principio de incerteza).

   
   argumentos:
   
   - data: serie temporal
   - fs: frequência de amostragem do encode original
   - plot: opcional, plota o espectrograma com parametros padrões para obter máxima frequência maior que a largura de banda.
  '''
 
  f,t,Sxx = signal.spectrogram(data,fs,nfft=fs/4,nperseg=fs/5,noverlap=fs/10,scaling='spectrum',mode='magnitude')
  freq = np.array([i for i in zip(t,Sxx.T) if np.shape(f[i[1]>np.std(i[1])])[0] != 0])
  
  if plot:
    plt.figure()
    plt.pcolormesh(t, f, np.log(Sxx+1e-13) )
    plt.ylabel('Frequency [Hz]')
    plt.xlabel('Time [sec]')
    plt.show()

  return [[ i[0], np.ceil(np.float(fs)/(2.0*np.max(f[i[1]>np.std(i[1])]) )) ] for i in freq]

In [0]:
####
## Spectrogram + Downsampling according to Nyquist-Shannon.
#

sampa = np.min(np.array(amostragem(np.array(measure)[:,0],fs, plot=True))[:,1])
sampb = np.min(np.array(amostragem(np.array(measure)[:,1],fs1, plot=True))[:,1])
fsa = (fs/int(sampa))
fsb=(fs1/int(sampb))

a = downsampling(np.array(measure)[:,0],sampa )
b = downsampling(np.array(measure)[:,1],sampb )

f,t,Sxx = signal.spectrogram(a,fsa,nfft=fsa/4,nperseg=fsa/5,noverlap=fsa/10,scaling='spectrum',mode='magnitude')
plt.figure()
plt.pcolormesh(t, f, np.log(Sxx+1e-13) )
plt.ylabel('Frequency [Hz]')
plt.xlabel('Time [sec]')

f,t,Sxx = signal.spectrogram(b,fsb,nfft=fsb/4,nperseg=fsb/5,noverlap=fsb/10,scaling='spectrum',mode='magnitude')
plt.figure()
plt.pcolormesh(t, f, np.log(Sxx+1e-13))
plt.ylabel('Frequency [Hz]')
plt.xlabel('Time [sec]')


plt.figure()
plt.plot(measure,label='original')
plt.figure()
plt.plot(a,label='downsampled')
plt.figure()
plt.plot(airnoise,label='original')
plt.figure()
plt.plot(b,label='downsampled')

audio.wavwrite(a,'/content/birdsdown.wav',fs=fsa,enc=enc)
#audio.wavwrite(b,'/content/b.wav',fs=fsb,enc=enc)